In [109]:
import numpy as np
import pandas as pd 
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
import math
import scipy

In [153]:
# formatting & aesthetics
font = {'family':'sans serif', 'size':10}
plt.rc('font', **font)
mpl.rcParams['xtick.labelsize'] = 10
mpl.rcParams['ytick.labelsize'] = 10
mpl.rcParams['legend.facecolor'] = 'white'
sns.set_style("white")

sns_pal = sns.color_palette('Set1', n_colors=8, desat=.4)
greys_pal = sns.color_palette('Greys', n_colors=9)

haz_dict = {'EQ':'earthquakes','HU':'wind events','SS':'storm surges','PF':'precipitation flooding'}

In [111]:
def get_expectation_values(df):
    
    #ix_levels = np.array(df.index.names)
    #ix_levels = np.array(ix_levels[((ix_levels!='affected_cat')&(ix_levels!='helped_cat'))])
    ix_levels = ['region','hazard','rp','hhid','quintile']
    #print(ix_levels)
    
    df_ev = pd.DataFrame(index=(df.sum(level=ix_levels)).index)
    
    for _col in df:
        if _col == 'pcwgt': df[['pcwgt']].sum(level=ix_levels)
        df_ev[_col] = df[['pcwgt',_col]].prod(axis=1).sum(level=ix_levels)/df['pcwgt'].sum(level=ix_levels)
        
    return df_ev
        

In [112]:
def load_hh_df(myCountry,pds_scenario='no'):
    
    in_file = '../../output_country/{}/poverty_duration_{}.csv'.format(myCountry,pds_scenario)
    master_ix = ['region','hazard','rp','hhid','affected_cat','helped_cat']

    df = pd.read_csv(in_file).set_index(master_ix).dropna(how='any')
    df = get_quintile_info(myCountry,df,master_ix+['quintile'])
    df = load_income_classification(myCountry,df,master_ix+['quintile'])
    return df.sort_index()

def get_quintile_info(myCountry,df,master_ix):
    quints = pd.read_csv('../../intermediate/{}/cat_info.csv'.format(myCountry),usecols=['hhid','quintile']).set_index('hhid')
    df = pd.merge(quints.reset_index(),df.reset_index(),on='hhid').set_index(master_ix)
    return df

def slice_on_pop(df,segment=None,poverty_type=None):
    if segment == 'poor': print('DANGER: not returning poor hh. is intended?')
    encode = {'poor':'pov',
              'vulnerable':'vul',
              'secure':'sec',
              'middleclass':'mc'}
    
    try: code = 't_{}_{}'.format(encode[segment],poverty_type)
    except: code = 'no_encoding'
        
    if segment == 'poor': 
        pop_slice = (False,True,True,True)
        df = df.loc[(df.ispoor==pop_slice[0])
                    &((df.isvulnerable==pop_slice[1])|(df.issecure==pop_slice[2])|(df.ismiddleclass==pop_slice[3]))]
    elif segment is not None:
        if segment == 'vulnerable': pop_slice = (False,True,False,False)
        if segment == 'secure': pop_slice = (False,False,True,False)
        if segment == 'middleclass': pop_slice = (False,False,False,True)
        df = df.loc[(df.ispoor==pop_slice[0])
                    &(df.isvulnerable==pop_slice[1])&(df.issecure==pop_slice[2])&(df.ismiddleclass==pop_slice[3])] 
    else: pass

    return df,code

def get_population(myCountry,segment):
    _cols = ['region','quintile','pcwgt','ispoor','ismiddleclass','issecure','isvulnerable']
    tot_pop = pd.read_csv('../../intermediate/{}/cat_info.csv'.format(myCountry),usecols=_cols).set_index(['region','quintile'])
    
    if segment != 'poor': tot_pop,_ = slice_on_pop(tot_pop,segment)
    else: tot_pop = tot_pop.loc[tot_pop.ispoor==True]
    
    tot_pop = tot_pop[['pcwgt']].sum(level=['region','quintile'])
    return tot_pop
    
def load_income_classification(myCountry,df,master_ix):
    is_poor = pd.read_csv('../../intermediate/{}/cat_info.csv'.format(myCountry),usecols=['hhid','ispoor','ismiddleclass','issecure','isvulnerable'])
    df = pd.merge(df.reset_index(),is_poor.reset_index(),on='hhid').set_index(master_ix)    
    return df


In [113]:
def average_over_rp(df,default_rp='default_rp',protection=None,return_probs=True):        
    """Aggregation of the outputs over return periods"""    
    if protection is None:
        protection=pd.Series(0,index=df.index)        

    #just drops rp index if df contains default_rp
    if default_rp in df.index.get_level_values('rp'):
        print('default_rp detected, dropping rp')
        return (df.T/protection).T.reset_index('rp',drop=True)
           
    df=df.copy().reset_index('rp')

    #computes frequency of each return period
    return_periods=np.unique(df['rp'].dropna())

    proba = pd.Series(np.diff(np.append(1/return_periods,0)[::-1])[::-1],index=return_periods) #removes 0 from the rps 

    #matches return periods and their frequency
    proba_serie=df['rp'].replace(proba).rename('prob')
    proba_serie1 = pd.concat([df.rp,proba_serie],axis=1)

    #handles cases with multi index and single index (works around pandas limitation)
    idxlevels = list(range(df.index.nlevels))

    if idxlevels==[0]:
        idxlevels =0
        
    #average weighted by proba
    averaged = df.mul(proba_serie,axis=0).sum(level=idxlevels).drop('rp',axis=1) # frequency times each variables in the columns including rp.


    if return_probs: return averaged,proba_serie1 #here drop rp.
    else: return averaged

In [114]:
#def main():
master_df = load_hh_df('PH')
master_df.head(500).to_csv('test.csv')

In [115]:
def plot_avg_time(df_ev,_haz,segment,fom,yr2mth=True):
    
    sf = 1.
    if yr2mth: sf = 12.  
        
    _ = slice(None)
    
    pop_df = get_population('PH',segment).sum(level='quintile').squeeze().to_frame(name='total_population')
    # indexed on quintile
    
    # 
    if segment != 'poor': df_ev[fom] = (10.019268 - df_ev[fom]).clip(lower=0)
    
    # plot weighted means
    # cycle over rps
    for _rp in df_ev.index.get_level_values(2).unique():
            
        _df = df_ev.loc[(_,_,[_rp],_,_),:]

        df_quint = _df[['pcwgt',fom]].prod(axis=1).sum(level='quintile').to_frame(name=fom)
        df_quint[fom] = df_quint[fom]/pop_df['total_population']
        plt.plot(df_quint.index,sf*df_quint[fom],color=greys_pal[6],lw=1.0,zorder=99)
                
        for _nreg, _reg in enumerate(df_ev.index.get_level_values(0).unique()):
            
            _regpop = get_population('PH',segment).reset_index('region')
            reg_pop = pd.DataFrame(index=get_population('PH',None).sum(level='quintile').index)
            reg_pop['tot_pop'] = _regpop.loc[_regpop.region==_reg].sum(level='quintile')
            reg_pop = reg_pop.fillna(-1)
            
            _dfreg = df_ev.loc[([_reg],_,[_rp],_,_),:]
            df_quint['{}_{}'.format(fom,_nreg)] = _dfreg[['pcwgt',fom]].prod(axis=1).sum(level='quintile')
            df_quint['{}_{}'.format(fom,_nreg)] = df_quint['{}_{}'.format(fom,_nreg)]/reg_pop['tot_pop']
            plt.plot(df_quint.index,sf*df_quint['{}_{}'.format(fom,_nreg)],color=greys_pal[4],alpha=0.25,lw=0.5,zorder=11)
            
        plt.fill_between(df_quint.index,sf*df_quint.min(axis=1),sf*df_quint.max(axis=1),alpha=0.1,color=greys_pal[3],zorder=10)
            
        plt.xlabel('Income quintile',labelpad=8,fontsize=10,linespacing=1.75)
        plt.ylabel('Time in poverty [{}]'.format('months' if yr2mth else 'years'),labelpad=10,fontsize=10,linespacing=1.75)

        plt.gca().tick_params(axis='both', which='major', pad=10,size=4)
        plt.xticks([1,2,3,4,5])
        plt.gca().set_xticklabels(['Poorest\nquintile','second','third','fourth','Wealthiest\nquintile'],ha='center',rotation=0)
        
        #plt.legend()
        sns.despine()
        plt.savefig('figures/duration/income_poverty_{}_{}.pdf'.format(_haz,_rp),format='pdf',bbox_inches='tight')
        plt.close('all')

In [116]:
keep_cols = ['pcwgt','c','dk0','dc_net_t0',
             'ispoor','ismiddleclass','isvulnerable','issecure',
             't_pov_cons','t_pov_inc',
             't_vul_cons','t_vul_inc',
             't_sec_cons','t_sec_inc', 
             't_mc_cons','t_mc_inc']

#loop over hazards:
for _haz in master_df.index.get_level_values(1).unique():
    if _haz != 'EQ': continue
        
    _ = slice(None)
    df = master_df.loc[(_,[_haz],_,_,_,_,_), keep_cols].sort_index()
    df,fom = slice_on_pop(df,'poor','cons')
    
    df_ev = get_expectation_values(df)
    
    plot_avg_time(df_ev,_haz,'poor',fom)
    
    
    #df_aal = average_over_rp(df,return_probs=False)

DANGER: not returning poor hh. is intended?


In [180]:
# plot average losses, absolute & relative
poverty_type = 'cons'

# OUTPUT
df_out = pd.DataFrame()

# loop over population segments
for segment in ['poor','vulnerable','secure','middleclass']:

    df_segment = get_population('PH',segment).sum(level='region').squeeze().to_frame(name='total_pop')
    df_segment['segment'] = segment
    
    # INPUT
    # region, hazard, rp, hhid, affected_cat, helped_cat, quintile
    # slice to select class (poor, vulnerable, secure, or middle class)
    cols = ['pcwgt','c','dk0','dc_net_t0','ispoor','issecure','isvulnerable','ismiddleclass']    
    if segment != 'poor': 
        df,_ = slice_on_pop(master_df,segment,poverty_type)
        df = df[cols]
    else: df = master_df.loc[master_df.ispoor==True,cols]
    
    df_segment['c'] = df[['pcwgt','c']].prod(axis=1).sum(level='region')/df['pcwgt'].sum(level='region')
    
    # loop over hazards
    for _haz in np.array(master_df.index.get_level_values(1).unique()):
        
        df_haz = df.loc[df.eval("hazard=='{}'".format(_haz)),:].sort_index()
        
        df_100 = df_haz.loc[df_haz.eval("rp==100")]
        df_segment['dk_100_'+_haz] = df_100[['dk0','pcwgt']].prod(axis=1).sum(level='region')
        df_segment['affpop_100_'+_haz] = df_100.loc[df_100.eval("affected_cat=='a'"),'pcwgt'].sum(level='region')
        
        df_aal = df_haz[['dk0','pcwgt']].prod(axis=1).sum(level=[0,2]).to_frame(name='dk_avg_'+_haz)
        df_aal['affpop_avg_'+_haz] = df_haz.loc[df_haz.eval("affected_cat=='a'"),'pcwgt'].sum(level=[0,2])
        df_segment[['dk_avg_'+_haz,'affpop_avg_'+_haz]] = average_over_rp(df_aal,return_probs=False)
              
    df_out = df_out.append(df_segment.fillna(0).reset_index().set_index(['region','segment']))
    
df_out.to_csv('df_out.csv')


# PLOT SAME
for nseg, segment in enumerate(['poor','vulnerable','secure','middleclass']):
    df_seg = df_out.loc[df_out.eval("segment=='{}'".format(segment))]
    
    # loop over hazards
    _bot = [0,0,0,0]
    for nhaz, _haz in enumerate(np.array(master_df.index.get_level_values(1).unique())):
        
        value = 1E2*df_seg['dk_avg_'+_haz].sum()/df_seg[['c','total_pop']].prod(axis=1).sum()
        
        plt.bar(nseg,value,bottom=_bot[nseg],
                color=sns_pal[1+nhaz],zorder=99,linewidth=0,edgecolor=None,width=0.70,alpha=0.6)
        
        if nseg == 0: plt.annotate(haz_dict[_haz],xy=(0.015,_bot[nseg]+0.01),color=greys_pal[8],fontsize=7,zorder=99)
        _bot[nseg]+=value
        if nhaz == 3: 
            _ = int(round(1E-2*_bot[nseg]*(df_seg[['c','total_pop']].prod(axis=1).sum()/df_seg['total_pop'].sum()),-1))
            if nseg == 0: 
                plt.annotate('Average loss:\n {} per cap'.format(_),xy=(nseg+0.7,_bot[nseg]),
                                       ha='right',va='bottom',color=greys_pal[8],fontsize=7,zorder=99)
            else: plt.annotate('{} pesos per cap'.format(_),xy=(nseg+0.7,_bot[nseg])
                               ,ha='right',va='bottom',color=greys_pal[8],fontsize=7,zorder=99)
        
plt.xlim(-0.2,3.9)
plt.xticks([0.35,1.35,2.35,3.35])
plt.gca().set_xticklabels(['Poor','Vulnerable','Secure','Middle class'],ha='center',rotation=0)

plt.ylabel('Average annual losses [% of total income]',labelpad=10)

sns.despine(left=True)
plt.savefig('figures/consumption_loss_rel_{}.pdf'.format(poverty_type),format='pdf',bbox_inches='tight')
plt.close('all')


# PLOT absolute losses
for nseg, segment in enumerate(['poor','vulnerable','secure','middleclass']):
    df_seg = df_out.loc[df_out.eval("segment=='{}'".format(segment))]
    
    # loop over hazards
    _bot = [0,0,0,0]
    for nhaz, _haz in enumerate(np.array(master_df.index.get_level_values(1).unique())):
        
        value = df_seg['dk_avg_'+_haz].sum()*1E-9
        
        plt.bar(nseg,value,bottom=_bot[nseg],
                color=sns_pal[1+nhaz],zorder=99,linewidth=0,edgecolor=None,width=0.70,alpha=0.6)
        
        if nseg == 3: plt.annotate(haz_dict[_haz],xy=(3.015,_bot[nseg]+0.2),color=greys_pal[8],fontsize=7,zorder=99)
        _bot[nseg]+=value
        if nhaz == 3: 
            if nseg == 0: plt.annotate('Total population: {} mil.'.format(round(df_seg['total_pop'].sum()*1E-6,1)),
                                       xy=(nseg+0.7,_bot[nseg]),ha='right',va='bottom',color=greys_pal[8],fontsize=7,zorder=99)
            else: plt.annotate('{} mil.'.format(round(df_seg['total_pop'].sum()*1E-6,1)),xy=(nseg+0.7,_bot[nseg]),
                               ha='right',va='bottom',color=greys_pal[8],fontsize=7,zorder=99)
plt.xlim(-0.2,3.9)
plt.xticks([0.35,1.35,2.35,3.35])
plt.gca().set_xticklabels(['Poor','Vulnerable','Secure','Middle class'],ha='center',rotation=0)

plt.ylabel('Average annual losses [bil. pesos per year]',labelpad=10)

sns.despine(left=True)
plt.savefig('figures/consumption_loss_abs_{}.pdf'.format(poverty_type),format='pdf',bbox_inches='tight')
plt.close('all')
        
        

In [65]:
# plot middle class stuff
for segment in ['secure','vulnerable','middleclass']:
    for poverty_type in ['cons','inc']:
        sf = 1.#12.

        # loop over hazards
        for _haz in np.array(master_df.index.get_level_values(1).unique()):
        
            # slice to select hazard
            _ = slice(None)
            df = master_df.loc[(_,[_haz],_,_,_,_,_), keep_cols].sort_index()
                
    
            # slice to select class (poor, vulnerable, secure, or middle class)
            df,fom = slice_on_pop(df,segment,poverty_type)
    
            # if we're looking at poverty, it's time *IN* poverty
            # otherwise, it's time outside of initial class
            if segment != 'poor': df[fom] = (10.019268-df[fom]).clip(lower=0)
    
            # get expectation values (sum over a/na/helped/not-helped)
            df_ev = get_expectation_values(df)
    
            # load entire population on class = segment
            pop_df = get_population('PH',segment).sum(level='region').squeeze().to_frame(name='pop')
            #print(pop_df.head())
    
            # get entire population of affected, for class we're considering
            pop_mc_aff = df[['pcwgt']].sum(level=['region','rp'])
            assert(pop_mc_aff.shape[0]>0)
        
            # total displacement [time, years]
            df_region = df[['pcwgt',fom]].prod(axis=1).sum(level=['region','rp']).to_frame(name=fom)
            #df_region[fom] /= pop_mc_aff['pcwgt']
            
            # average over rp
            df_region_aal = average_over_rp(df_region,return_probs=False)
            # ^ this is displacement*time * RESULT*
            df_region_aal['frac_{}'.format(fom)] = df_region_aal[fom]/pop_df['pop']
            # this is fraction of personXyears spent displaced

            df_region_aal['pop_aff_{}'.format(fom)] = average_over_rp(pop_mc_aff,return_probs=False)
            # ^ this is average annual affected population
    
            # Sort ascending
            df_region_aal = df_region_aal.sort_values(fom,ascending=True).reset_index()
    
            # plot 
            plt.bar(df_region_aal.index,1E-3*sf*df_region_aal[fom],color=sns_pal[1],lw=1.0,zorder=99,width=0.8,alpha=0.3)
    
            # label barplot    
            rects = plt.gca().patches
            for nrect, rect in enumerate(rects):
                
                _val = df_region_aal.iloc[nrect]['frac_{}'.format(fom)]
                _max = df_region_aal['frac_{}'.format(fom)].max()
                _col = sns_pal[0] if (_val == _max) else greys_pal[8]
                _wgt = 'bold'  if (_val == _max) else 'normal'
                
                try:
                    #print(nrect)
                    # total population of middle class +
                    #plt.annotate('{}'.format(int(1E-3*pop_mc_aff.iloc[nrect].squeeze())),
                    #             xy=(rect.get_x()+rect.get_width()/2,rect.get_height()),va='bottom',ha='center')
            
                    # fraction of person-years
                    plt.annotate('{}%'.format(round(float(1E2*df_region_aal.iloc[nrect]['frac_{}'.format(fom)].squeeze()),1)),
                                 xy=(rect.get_x()+rect.get_width()/2,rect.get_height()),va='bottom',ha='center',fontsize=8,color=_col,weight=_wgt)
                except: pass
    
            plt.ylabel('Average annual displacement\namong {} [,000 person-years]'.format(segment.replace('middle','middle ')),labelpad=8,fontsize=10,linespacing=1.75)
    
            plt.xticks(df_region_aal.index+0.4)
        
            plt.gca().set_xticklabels(df_region_aal.region,ha='center',rotation=90)
            plt.grid(True,axis='y')
    
            sns.despine(left=True)
            plt.savefig('figures/{}_{}_{}.pdf'.format(poverty_type,segment,_haz),format='pdf',bbox_inches='tight')
            plt.close('all')
    
            #df_aal = average_over_rp(df,return_probs=False)